# Dataset

In [1]:
!pip install -q mediapipe

# Skrypt

In [5]:
import cv2
import mediapipe as mp
import sqlite3
import time
import numpy as np

# Inicjalizacja MediaPipe
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.7)

# Dane wejściowe
user_id = input("Podaj ID użytkownika: ").strip()
label = input("Podaj literę języka migowego (np. A): ").upper().strip()
num_samples = int(input("Podaj liczbę próbek do nagrania: "))

# Baza danych
db_path = 'PJM-sign-language.db'
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Nazwa tabeli na podstawie litery
table_name = f"{label}"

# Tworzenie tabeli jeśli nie istnieje
cursor.execute(f'''
CREATE TABLE IF NOT EXISTS {table_name} (
    user_id TEXT,
    label TEXT,
    sample_id INTEGER,
    frame INTEGER,
    ''' + ', '.join([f'landmark_{i}_{axis} REAL' for i in range(1, 22) for axis in ('x', 'y', 'z')]) + ''')
''')
conn.commit()

# Kamera
cap = cv2.VideoCapture(0)
sample_counter = 0
FRAME_RATE = 15
DURATION_SEC = 2
TOTAL_FRAMES = FRAME_RATE * DURATION_SEC

print(f"\nNaciśnij SPACJĘ aby nagrać próbkę (15 FPS, 2 sekundy) dla litery '{label}'. 'q' aby zakończyć.")

while sample_counter < num_samples:
    # Pobierz ostatni sample_id
    cursor.execute(f"SELECT MAX(sample_id) FROM {table_name} WHERE user_id = ? AND label = ?", (user_id, label))
    result = cursor.fetchone()
    next_sample_id = (result[0] + 1) if result[0] is not None else 0

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Nie można odczytać klatki z kamery.")
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

        display_text = f"User: {user_id}  Label: {label}  Sample: {sample_counter+1}/{num_samples}"
        cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2, cv2.LINE_AA)
        cv2.putText(frame, "Nacisnij SPACJE aby nagrywac", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2, cv2.LINE_AA)

        cv2.imshow("Kamera", frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord(' '):  # Spacja - rozpoczęcie nagrania
            print("Rozpoczynanie nagrywania...")
            frame_count = 0

            while frame_count < TOTAL_FRAMES:
                ret, frame = cap.read()
                if not ret:
                    print("Błąd odczytu z kamery.")
                    break

                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                results = hands.process(rgb_frame)

                draw_frame = frame.copy()
                if results.multi_hand_landmarks:
                    for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(draw_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                    landmarks = results.multi_hand_landmarks[0].landmark
                    row = [user_id, label, next_sample_id, frame_count]
                    for lm in landmarks[:21]:
                        row += [lm.x, lm.y, lm.z]
                    placeholders = ','.join(['?'] * len(row))
                    cursor.execute(f"INSERT INTO {table_name} VALUES ({placeholders})", row)
                    conn.commit()

                    frame_count += 1  # Zliczaj tylko gdy jest dłoń

                cv2.putText(draw_frame, f"NAGRYWANIE {frame_count}/{TOTAL_FRAMES}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2, cv2.LINE_AA)

                cv2.imshow("Kamera", draw_frame)
                cv2.waitKey(1)
                time.sleep(1 / FRAME_RATE)

            print(f"Zakończono nagrywanie próbki {next_sample_id}.")
            sample_counter += 1
            break

        elif key == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            conn.close()
            print("Zakończono.")
            exit()

cap.release()
cv2.destroyAllWindows()
conn.close()
print(f"Zakończono nagrywanie. Dane zapisane do '{db_path}'.")


W0000 00:00:1747144267.130169   67622 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1747144267.158135   67622 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.



Naciśnij SPACJĘ aby nagrać próbkę (15 FPS, 2 sekundy) dla litery 'Z_X'. 'q' aby zakończyć.
Rozpoczynanie nagrywania...
Zakończono nagrywanie próbki 0.
Rozpoczynanie nagrywania...
Zakończono nagrywanie próbki 1.
Zakończono nagrywanie. Dane zapisane do 'PJM-sign-language.db'.
